# Distribution analysis

* Calculate the number of overlapping addresses between the Aelin pool and the staker/trader eligibility
* Calculate the amount of KWENTA assigned to these addresses


In [3]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal


In [4]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/aelin-xyz/aelin-optimism'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-kovan-main'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [5]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)

def clean_df(df, decimal_cols):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [13]:
# queries
poolQuery = gql("""
query poolAccepters(
    $last_id: ID!
  ) {
    acceptDeals (
      where: {
        id_gt: $last_id
        poolAddress: "0x20369baa917bd1867bdafc24d72458ac777c9a2c"
      }
      first: 1000
     ) {
      id
      purchaser
    }  
}
""")


### Get the data

In [29]:
# read the raw files
df_stakers = pd.read_csv('./data/stakers.csv', header=None, names=['address', 'tokens'])
df_traders = pd.read_csv('./data/traders.csv', header=None, names=['address', 'tokens'])

# clean the columns
df_stakers['tokens'] = df_stakers['tokens'].apply(convertDecimals)
df_traders['tokens'] = df_traders['tokens'].apply(convertDecimals)

# combine them
df_token = df_stakers.merge(df_traders, how='outer', on='address', suffixes=['_stakers', '_traders']).fillna(0)
df_token['tokens'] = df_token['tokens_stakers'] + df_token['tokens_traders']


,address,tokens_stakers,tokens_traders,tokens
0,0x99f4176ee457afedffcb1839c7ab7a030a5e4a92,14716.225092471561099298,0,14716.225092471561099298
1,0x8ca24021e3ee3b5c241bbfcee0712554d7dc38a1,13346.406858307671844081,0,13346.406858307671844081
2,0x27cc4d6bc95b55a3a981bf1f1c7261cda7bb0931,6492.137841815433571518,5.254409792085848423,6497.392251607519419941
3,0xa5f7a39e55d7878bc5bd754ee5d6bd7a7662355b,6469.485039962326489519,5.254409792085848423,6474.739449754412337942
4,0xc8c2b727d864cc75199f5118f0943d2087fb543b,5709.397409534841170711,5.254409792085848423,5714.651819326927019134
...,...,...,...,...
5242,0x3c6a22e43fc6cd33dcda8af3bb9926c71e24b44b,0,5.254409792085848423,5.254409792085848423
5243,0x4f67c86d8919760cab9cbad538e0737623b4abf6,0,5.254409792085848423,5.254409792085848423
5244,0xc8f7060244d9c0a39b5bf2739dc3acf984237192,0,5.254409792085848423,5.254409792085848423
5245,0xbef5dab691ec9ba90bdfbada785e11c38844405a,0,5.254409792085848423,5.254409792085848423


In [14]:
pool_params = {
    'last_id': ''
}

df_pool = await run_recursive_query(poolQuery, pool_params, 'acceptDeals')
df_pool

,id,purchaser
0,0x0094aa2e3daf83b3ab27e6e6c2b204e5c24a5b8435d3...,0x865d7eb5db37cc02ec209dd778f4e3851a421a20
1,0x00c987f72f85f58b0c609b510e24a8114ce1150d0748...,0x6d82154d1f2ee7a37e9822647018ef97cec4cd23
2,0x014b6a41c2b6a133cb9c5e350121f40d41882dd66bb5...,0x7bb9f2396efba69b62efd183b06bd110dbf42152
3,0x02e7ef7f26507162d6119f5244a4efd7ac3ddce875e4...,0x401bf7e9e4d1f95305e4be928957ec6dcc1d72c7
4,0x02ec37b8e7a2cb4d9cee23e1161073a2858189937879...,0x9050c755691e9089e08adee29462bab6b338def0
...,...,...
602,0xfd2ae7d35e6f66fda90df1ae1e25618b8ad48df83664...,0x865fc5c542280c9dba0d559f67112179e20574d2
603,0xfe048915be81f5ee6803ae39ff4c75494e2b7b949694...,0xfbbedcc25e12dab1451e34b52b7b09419410ffc0
604,0xfe9eaf159b455ef4cb7e889de0b9ef0ac619c9968b00...,0xd1421ae08b24f5b24fa97980341dabcadeed3873
605,0xff41e9d7eb30adb6d8889c64974794db86189015d4ad...,0x23f198b0c23cf1bcce7b38a1fde27ae76541c500


In [40]:
# get unique pool addresses
pool_addresses = df_pool['purchaser'].unique()
df_token['pool_purchaser'] = df_token['address'].isin(pool_addresses)


In [42]:
df_token.pivot_table(index='pool_purchaser', values='tokens', aggfunc='sum')

,tokens
pool_purchaser,
False,100574.348689866173663995
True,9106.201310133707439200


In [44]:
print("total tokens: ", df_token['tokens'].sum())

total tokens:  109680.549999999881103195
